<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Natural-language-to-SQL" data-toc-modified-id="Natural-language-to-SQL-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Natural language to SQL</a></span></li><li><span><a href="#Prompt-without-Shots." data-toc-modified-id="Prompt-without-Shots.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prompt without Shots.</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></div>

# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [19]:
from google.colab import userdata
API_KEY = userdata.get('HF')

In [19]:
#https://colab.research.google.com/drive/1NlJGGCw1T9yloUk7OBBGYuwyfyctaGlW#scrollTo=oHSF1l3xNscJ

In [20]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-kovw36et
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-kovw36et
  Resolved https://github.com/huggingface/accelerate.git to commit 1f7a79b428749f45187ec69485f2c966fe21926e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-98lf8l26
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-98lf8l26
  Resolved https://github.com/huggingface/transformers.git to commit 1fd60fec75b2f75432cca48ff79d606e084a9bc2
  Installing build dependencies ... done
  Getting requirements to build whe

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate



In [22]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [23]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [24]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

This function wraps the call to *model.generate*

In [32]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [33]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar

    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - course_ID: int
    - enrollment_date: date
    - grade: varchar

    Table 4: Departments
    Table: departments
    Fields:
    - department_ID: int
    - name: varchar
    - building: varchar
    - budget: float

    Table 5: Research_Projects
    Table: research_projects
    Fields:
    - project_ID: int
    - title: varchar
    - lead_instructor_ID: int
    - start_date: date
    - end_date: date
    - funding_amount: float


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [34]:
sp_nl2sql = sp_nl2sql.format(question="how can i find the emails and names of hires in the last 6 months?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar

    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - course_ID: int
    - enrollment_date: date
    - grade: varchar

    Table 4: Departments
    Table: departments
    Fields:
    - depar

In [35]:
# Tokenize the input sentences
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')

# Generate response from the model
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)

# Decode the response to SQL query
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar

    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - course_ID: int
    - enrollment_date: date
    - grade: varchar

    Table 4: Departments
    Table: departments
    Fields:
    - depar

In [36]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [37]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT instructors.name, instructors.email FROM instructors JOIN enrollments ON instructors.instructor_ID = enrollments.instructor_ID WHERE enrollments.enrollment_date >= (CURRENT_DATE - interval '6 months') GROUP BY instructors.name, instructors.email ORDER BY instructors.name NULLS LAST;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [41]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar


    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - course_ID: int
    - enrollment_date: date
    - grade: varchar

    Table 4: Departments
    Table: departments
    Fields:
    - department_ID: int
    - name: varchar
    - building: varchar
    - budget: float

    Table 5: Research_Projects
    Table: research_projects
    Fields:
    - project_ID: int
    - title: varchar
    - lead_instructor_ID: int
    - start_date: date
    - end_date: date
    - funding_amount: float

### Question:Response One Shot
    how can i find the emails and names of hires in the last 6 months?:
    SELECT instructors.name, instructors.email
    FROM instructors
    JOIN enrollments ON instructors.instructor_ID = enrollments.instructor_ID
    WHERE enrollments.enrollment_date >= (CURRENT_DATE - interval '6 months')
    GROUP BY instructors.name, instructors.email
    ORDER BY instructors.name NULLS LAST;


    `{question}`:
    ```sql3
    """


In [42]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))




    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar

    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - course_ID: int
    - enrollment_date: d

In [43]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [44]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT departments.name, MAX(research_projects.funding_amount) AS max_funding 
     FROM research_projects 
     JOIN lead_instructors ON research_projects.lead_instructor_id = lead_instructors.instructor_id 
     JOIN instructors ON lead_instructors.instructor_id = instructors.instructor_id 
     JOIN departments ON instructors.department = departments.department_id 
     GROUP BY departments.name 
     ORDER BY max_funding DESC NULLS LAST LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [46]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar
    - students_missing: varchar


    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - course_ID: int
    - enrollment_date: date
    - grade: varchar

    Table 4: Departments
    Table: departments
    Fields:
    - department_ID: int
    - name: varchar
    - building: varchar
    - budget: float

    Table 5: Research_Projects
    Table: research_projects
    Fields:
    - project_ID: int
    - title: varchar
    - lead_instructor_ID: int
    - start_date: date
    - end_date: date
    - funding_amount: float

### Question:Response One Shot
    how can i find the emails and names of hires in the last 6 months?:
    SELECT instructors.name, instructors.email
    FROM instructors
    JOIN enrollments ON instructors.instructor_ID = enrollments.instructor_ID
    WHERE enrollments.enrollment_date >= (CURRENT_DATE - interval '6 months')
    GROUP BY instructors.name, instructors.email
    ORDER BY instructors.name NULLS LAST;

    Return The name of the best paid employee:
    I cannot find the salaries in the schema but I can find the best funded employee
    SELECT departments.name, MAX(research_projects.funding_amount) AS max_funding
    FROM research_projects
    JOIN lead_instructors ON research_projects.lead_instructor_id = lead_instructors.instructor_id
    JOIN instructors ON lead_instructors.instructor_id = instructors.instructor_id
    JOIN departments ON instructors.department = departments.department_id
    GROUP BY departments.name
    ORDER BY max_funding DESC NULLS LAST LIMIT 1;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [47]:
sp_nl2sql3 = sp_nl2sql3b.format(question="find the number of timese each student missed a class")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar
    - students_missing: varchar
  

    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - cours

In [48]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [49]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT students_missing.student_id, students_missing.course_id, COUNT(students_missing.student_id) AS number_of_missed_classes 
     FROM students_missing 
     GROUP BY students_missing.student_id, students_missing.course_id 
     ORDER BY number_of_missed_classes DESC NULLS LAST;


#Now the question in spanish.


In [50]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Encuentre el número de veces que cada estudiante faltó a una clase")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    Schema:
    Table 1: Courses
    Table: courses
    Fields:
    - course_ID: int
    - course_name: varchar
    - description: text
    - credits: int
    - department: varchar
    - students_missing: varchar
  

    Table 2: Instructors
    Table: instructors
    Fields:
    - instructor_ID: int
    - name: varchar
    - email: varchar
    - department: varchar
    - hire_date: date

    Table 3: Enrollments
    Table: enrollments
    Fields:
    - enrollment_ID: int
    - student_ID: int
    - cours

In [51]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [52]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT students.student_id, COUNT(enrollments.enrollment_id) AS num_enrollments 
     FROM students 
     LEFT JOIN enrollments ON students.student_id = enrollments.student_id 
     WHERE enrollments.enrollment_id IS NULL 
     GROUP BY students.student_id 
     ORDER BY num_enrollments DESC NULLS LAST;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [53]:
# There were some hallucinations when it came to asking questions requesting tasks that are not possible.
# I tried training the model to also explain if it's not possible to do a task and give a possible solution but that didn't work